In [12]:
from tmu.tsetlin_machine import TMCoalescedClassifier
import numpy as np 
import pickle
import torch
import tqdm
from sklearn.model_selection import train_test_split
from collections import Counter, defaultdict
!export OMP_NUM_THREADS=10
n_clauses_per_class = 8000
treshold = int(n_clauses_per_class * 0.8)
s_param = 11.40000000000001
# s_param = 5

"s henger sammen hvor ofte du ser pattern"
"lav s sier at man ser et pattern ofte, og kan glemme dette"
"belonning er s/(s-1)"
"random penalty er 1/s"

'random penalty er 1/s'

In [13]:
cuda_tm = TMCoalescedClassifier(n_clauses_per_class, treshold, s_param,  platform='CUDA', boost_true_positive_feedback=0)


In [3]:
game_amount = 100000
# tsetlin_filename, self_can_bet_index= "/home/jaoi/master22/pet_for_sale/winning_games_db/{}_tsetlined_games.pkl".format(game_amount), 17
tsetlin_filename, self_can_bet_index = "/home/jaoi/master22/pet_for_sale/winning_games_db/4PPO_{}_tsetlined_games.pkl".format(game_amount), 21

with open(tsetlin_filename, "rb") as fp:
        games = pickle.load(fp)

In [4]:
# preprossesing
where_can = []
for i, game in enumerate(games):
    if game[0][self_can_bet_index] == 1:
        where_can.append(i)
indexes = np.array(where_can)

games_can_bid = games[indexes]

def remove_duplicate_pairs(games):
    new_games = []
    for i,game in enumerate(games):
        int_arr = game[0].astype(int)
        complete_arr = np.append(int_arr, game[1])
        new_games.append(complete_arr)

    new_games = np.array(new_games)
    
    x = np.random.rand(new_games.shape[1])
    y = new_games.dot(x)
    unique, index = np.unique(y, return_index=True)
    unique_xy = new_games[index]
    
    unique_pairs = []
    for game in unique_xy:
        last, rest = game[-1], game[:-1]
        unique_pairs.append(np.array((rest.astype(np.float32), last), dtype=object))
    return np.array(unique_pairs)
unique_pairs = remove_duplicate_pairs(games_can_bid)

ysu = Counter()
for game in unique_pairs:
    ysu[game[1]] += 1
    
need_of_oversample = [tup for tup in ysu.most_common() if tup[1] < 2000]

def oversample(info_tup, desired_amount, source_games):
    bob = []
    for game in source_games:
        if game[1] == info_tup[0]:
            bob.append(game)
    indexes = np.random.randint(0, info_tup[1], desired_amount - info_tup[1])
    temp = []
    for ind in indexes:
        temp.append(bob[ind])
    return np.array(temp)

unique_oversample = np.copy(unique_pairs)
temp = []
for overnd in need_of_oversample:
    temp.extend(oversample(overnd, 2000, unique_pairs))
unique_oversample = np.concatenate((unique_oversample, np.array(temp)), axis=0)

uo = Counter()
for game in unique_oversample:
    uo[game[1]] += 1
uo

Counter({2: 43489,
         0: 134296,
         3: 38945,
         4: 79133,
         5: 27761,
         6: 55871,
         8: 34897,
         1: 2000,
         9: 12721,
         7: 19703,
         10: 13970,
         11: 5188,
         12: 3549,
         13: 2000,
         14: 2000,
         15: 2000,
         16: 2000})

In [5]:
x =np.array([game[0] for game in unique_oversample[:, :1]])
y =np.array([game[0] for game in unique_oversample[:, 1:]])


In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.2)

In [ ]:
# split out the different classes, and check if some of the classes are making it harder for the TM

In [85]:
# train_i = int(unique_oversample.shape[0] * 0.9)
# np.random.shuffle(unique_oversample)
# x_train =np.array([game[0] for game in unique_oversample[:train_i, :1]])
# x_test =np.array([game[0] for game in unique_oversample[train_i:, :1]])

# y_train =np.array([game[0] for game in unique_oversample[:train_i, 1:]])
# y_test =np.array([game[0] for game in unique_oversample[train_i:, 1:]])

In [14]:
epochs = 10
# for epoch in range(epochs):
#     tm.fit(x_train, y_train)

# print("Accuracy tm:", 100*(tm.predict(x_test) == y_test).mean())

for epoch in range(epochs):
    cuda_tm.fit(x_train, y_train)

    print("Epoch {} Accuracy tm:".format(epoch), 100*(cuda_tm.predict(x_test) == y_test).mean())

Epoch 0 Accuracy tm: 79.25029977581983
Epoch 1 Accuracy tm: 79.72472759501589
Epoch 2 Accuracy tm: 79.79875918878056
Epoch 3 Accuracy tm: 79.96142015536208
Epoch 4 Accuracy tm: 79.72055680100098
Epoch 5 Accuracy tm: 79.6986601324227
Epoch 6 Accuracy tm: 78.973984672332
Epoch 7 Accuracy tm: 79.06365674365257
Epoch 8 Accuracy tm: 78.62050987956832
Epoch 9 Accuracy tm: 78.81757989677286


In [33]:
def get_name_of_feature_from_3_rounds(feature_nr):
    # for own player with 4 bits representing the cards as max number is 11
    reverse_4_bits = [3, 2, 1, 0]
    reverse_5_bits = [4, 3, 2, 1, 0]
    reverse_2_bits = [1, 0]
    if feature_nr in [0,1,2,3,4]:
        return "own coins bit {}".format(2**(reverse_5_bits[feature_nr]))
    elif feature_nr in [5,6,7,8]:
        return "own cards 1. bit {}".format(2**reverse_4_bits[feature_nr - 5])
    elif feature_nr in [9,10,11,12]:
        return "own cards 2. bit {}".format(2**reverse_4_bits[feature_nr - 9])
    elif feature_nr in [13,14,15,16]:
        return "own cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 13])
    elif feature_nr == 17:
        return "self can bid bit"
    elif feature_nr in [18,19,20,21,22]:
        return "current own bid {}".format(2**reverse_5_bits[feature_nr - 18])
    elif feature_nr in [23,24,25,26,27]:
        return "resulting own coins if fold {}".format(2**reverse_5_bits[feature_nr - 23])
    elif feature_nr in [28,29,30,31]:
        return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 28])
    elif feature_nr in [32,33]:
        return "self next turn order {}".format(2**reverse_2_bits[feature_nr - 32])
    # for next player in line
    elif feature_nr in [34, 35, 36, 37, 38]:
        return "next player coins bit {}".format(2**(reverse_5_bits[feature_nr - 34]))
    elif feature_nr in [39, 40, 41, 42]:
        return "next player cards 1. bit {}".format(2**reverse_4_bits[feature_nr - 39])
    elif feature_nr in [43, 44, 45, 46]:
        return "next player cards 2. bit {}".format(2**reverse_4_bits[feature_nr - 43])
    elif feature_nr in [47, 48, 49, 50]:
        return "next player cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 47])
    elif feature_nr == 51:
        return "next can bid bit"
    elif feature_nr in [52, 53, 54, 55, 56]:
        return "current next player bid {} bit".format(2**reverse_5_bits[feature_nr - 52])
    elif feature_nr in [57, 58, 59, 60, 61]:
        return "resulting next player coins if fold {}".format(2**reverse_5_bits[feature_nr - 57])
    elif feature_nr in [62, 63, 64, 65]:
        return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 62])
    elif feature_nr in [66, 67]:
        return "next player turn order {}".format(2**reverse_2_bits[feature_nr - 66])
    # for last player in line
    elif feature_nr in [68, 69, 70, 71, 72]:
        return "last player coins bit {}".format(2**(reverse_5_bits[feature_nr - 68]))
    elif feature_nr in [73, 74, 75, 76]:
        return "last player cards 1. bit {}".format(2**reverse_4_bits[feature_nr - 73])
    elif feature_nr in [77, 78, 79, 80]:
        return "last player cards 2. bit {}".format(2**reverse_4_bits[feature_nr - 77])
    elif feature_nr in [81, 82, 83, 84]:
        return "last player cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 81])
    elif feature_nr == 85:
        return "last player can bid"
    elif feature_nr in [86, 87, 88, 89, 90]:
        return "current next player bid {}".format(2**reverse_5_bits[feature_nr - 86])
    elif feature_nr in [91, 92, 93, 94, 95]:
        return "resulting next player coins if fold {}".format(2**reverse_5_bits[feature_nr - 91])
    elif feature_nr in [96, 97, 98, 99]:
        return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 96])
    elif feature_nr in [100, 101]:
        return "last player turn order {}".format(2**reverse_2_bits[feature_nr - 100])
    # remainder stats
    elif feature_nr in [102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112]:
        return "{}. largest in stack".format(feature_nr - 101)
    elif feature_nr in [113, 114, 115, 116]:
        return "last player cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 113])
    elif feature_nr in [117, 118, 119, 120]:
        return "last player cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 117])
    elif feature_nr in [121, 122, 123, 124]:
        return "last player cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 121])
    elif feature_nr == 125:
        return "board_std"
    return "not implemented"

In [ ]:
def get_name_of_feature_from_4_rounds(feature_nr):
    # for own player with 4 bits representing the cards as max number is 11
    reverse_4_bits = [3, 2, 1, 0]
    reverse_5_bits = [4, 3, 2, 1, 0]
    reverse_2_bits = [1, 0]
    if feature_nr in [0,1,2,3,4]:
        return "own coins bit {}".format(2**(reverse_5_bits[feature_nr]))
    elif feature_nr in [5,6,7,8]:
        return "own nr 1 card bit {}".format(2**reverse_4_bits[feature_nr - 5])
    elif feature_nr in [9,10,11,12]:
        return "own nr 2 card bit".format(2**reverse_4_bits[feature_nr - 9])
    elif feature_nr in [13,14,15,16]:
        return "own nr 3 card bit".format(2**reverse_4_bits[feature_nr - 13])
    elif feature_nr in [17,18,19,20]:
        return "own nr 4 card bit".format(2**reverse_4_bits[feature_nr - 17])
    elif feature_nr == 21:
        return "self can bid bit"
    elif feature_nr in [22,23,24,25,26]:
        return "current own bid {}".format(2**reverse_5_bits[feature_nr - 22])
    elif feature_nr in [27,28,29,30,31]:
        return "resulting own coins if fold {}".format(2**reverse_5_bits[feature_nr - 27])
    elif feature_nr in [32,33,34,35]:
        return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 32])
    elif feature_nr in [36,37]:
        return "self next turn order {}".format(2**reverse_2_bits[feature_nr - 36])
    # for next player in line
    elif feature_nr in [38,39,40,41,42]:
        return "own coins bit {}".format(2**(reverse_5_bits[feature_nr - 38]))
    elif feature_nr in [43,44,45,46]:
        return "own nr 1 card bit {}".format(2**reverse_4_bits[feature_nr - 43])
    elif feature_nr in [47,48,49,50]:
        return "own nr 2 card bit".format(2**reverse_4_bits[feature_nr - 47])
    elif feature_nr in [51,52,53,54]:
        return "own nr 3 card bit".format(2**reverse_4_bits[feature_nr - 51])
    elif feature_nr in [55,56,57,58]:
        return "own nr 3 card bit".format(2**reverse_4_bits[feature_nr - 55])
    elif feature_nr == 59:
        return "next can bid bit"
    elif feature_nr in [60, 61, 62, 63, 64]:
        return "current next player bid {} bit".format(2**reverse_5_bits[feature_nr - 60])
    elif feature_nr in [65, 66, 67, 68, 69]:
        return "resulting next player coins if fold {}".format(2**reverse_5_bits[feature_nr - 65])
    elif feature_nr in [70, 71, 72, 73]:
        return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 70])
    elif feature_nr in [74, 75]:
        return "next player turn order {}".format(2**reverse_2_bits[feature_nr - 74])
    # for last player in line
    elif feature_nr in [76,77,78,79,80]:
        return "own coins bit {}".format(2**(reverse_5_bits[feature_nr - 76]))
    elif feature_nr in [81,82,83,84]:
        return "own nr 1 card bit {}".format(2**reverse_4_bits[feature_nr - 81])
    elif feature_nr in [85,86,87,88]:
        return "own nr 2 card bit".format(2**reverse_4_bits[feature_nr - 85])
    elif feature_nr in [89,90,91,92]:
        return "own nr 3 card bit".format(2**reverse_4_bits[feature_nr - 89])
    elif feature_nr in [93,94,95,96]:
        return "own nr 3 card bit".format(2**reverse_4_bits[feature_nr - 93])
    elif feature_nr == 97:
        return "last player can bid"
    elif feature_nr in [98, 99, 100, 101, 102]:
        return "current next player bid {}".format(2**reverse_5_bits[feature_nr - 98])
    elif feature_nr in [103, 104, 105, 106, 107]:
        return "resulting next player coins if fold {}".format(2**reverse_5_bits[feature_nr - 103])
    elif feature_nr in [108, 109, 110, 111]:
        return "resulting card on fold {}".format(2**reverse_4_bits[feature_nr - 108])
    elif feature_nr in [112, 113]:
        return "last player turn order {}".format(2**reverse_2_bits[feature_nr - 112])
    # remainder stats
    elif feature_nr in [114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128]:
        return "{}. largest in stack".format(feature_nr - 114)
    elif feature_nr in [129, 130, 131, 132]:
        return "last player cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 129])
    elif feature_nr in [133, 134, 135, 136]:
        return "last player cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 133])
    elif feature_nr in [137, 138, 139, 140]:
        return "last player cards 3. bit {}".format(2**reverse_4_bits[feature_nr - 137])
    elif feature_nr == 141:
        return "board_std"
    return "not implemented"

In [26]:
def show_coalesced_clause_3r(tm, clause):
    number_of_features = 126
    print("Clause #%d: " % (clause), end=' ')
    l = []
    for k in range(number_of_features*2):
        if tm.get_ta_action(clause, k) == 1:
            if k < number_of_features:
                l.append(" "+get_name_of_feature_from_3_rounds(k))
            else:
                l.append("¬"+get_name_of_feature_from_3_rounds(k-number_of_features))
    return " ∧ ".join(l)

def show_coalesced_clause_4r(tm, clause):
    number_of_features = 142
    print("Clause #%d: " % (clause), end=' ')
    l = []
    for k in range(number_of_features*2):
        if tm.get_ta_action(clause, k) == 1:
            if k < number_of_features:
                l.append(" "+get_name_of_feature_from_4_rounds(k))
            else:
                l.append("¬"+get_name_of_feature_from_4_rounds(k-number_of_features))
    return " ∧ ".join(l)

def show_coalesced_weight(tm, the_class, weight):
    mw = tm.weight_banks[the_class].get_weights()
    print(mw[weight])


In [35]:
def show_activated_clauses_location(tm, x):
    c_out = tm.transform(x.reshape(1, -1))
    clauses_out = np.where(c_out == 1)[1]
#     moves_and_scores = []
#     classes = len(tm.weight_banks)
#     for clas in range(classes):
#         mw = cuda_tm.weight_banks[clas].get_weights()
#         # print(np.argpartition(mw, -5)[-5:])
#         moves_and_scores.append((clas, np.dot(c_out, mw)))
#     top_moves = sorted(moves_and_scores, reverse=True, key=lambda x:x[1])

#     if top_moves[0][0] != tm.predict(x.reshape(1, -1))[0]:
#         print("OH OH <STINKY!> {} {}".format(top_moves[:5], tm.predict(x.reshape(1, -1))[0]))
#         return True
#     else:
#         False
    return clauses_out
show_activated_clauses_location(cuda_tm, x_test[0])

array([  4,   7,  15,  16,  22,  36,  41,  51,  52,  69, 114, 117, 123,
       129, 141, 142, 185, 191, 194])

In [39]:
def show_most_important_clauses_per_class(tm):
    classes = len(tm.weight_banks)
    for clas in range(classes):
        mw = tm.weight_banks[clas].get_weights()
        #largest for
        print("clauses for class {}". format(clas))
        for c in np.argpartition(mw, -5)[-5:]:
            print("{} weight, rule: {}".format(mw[c], show_coalesced_clause(tm, c)))
        print()
        #largest against
        print(np.argpartition(mw, 5)[:5])
show_most_important_clauses_per_class(cuda_tm)

clauses for class 0
Clause #179:  38 weight, rule:  current next player bid 8 bit
Clause #12:  42 weight, rule:  current next player bid 4 bit ∧  next player turn order 1
Clause #62:  74 weight, rule:  current next player bid 4 bit
Clause #87:  66 weight, rule:  own coins bit 16 ∧  next player coins bit 16 ∧ ¬own coins bit 8 ∧ ¬own coins bit 1 ∧ ¬current next player bid 2 bit
Clause #174:  45 weight, rule:  current next player bid 8

[13 72 76 67 49]
clauses for class 1
Clause #145:  59 weight, rule:  last player turn order 2 ∧ ¬current own bid 8
Clause #16:  62 weight, rule:  last player cards 1. bit 4 ∧  resulting card on fold 8 ∧ ¬current next player bid 1
Clause #193:  68 weight, rule: ¬own coins bit 16 ∧ ¬own coins bit 4 ∧ ¬last player cards 3. bit 4
Clause #46:  92 weight, rule: ¬current own bid 2 ∧ ¬current own bid 1 ∧ ¬current next player bid 4 bit ∧ ¬current next player bid 1 bit ∧ ¬current next player bid 1 ∧ ¬resulting card on fold 4
Clause #99:  80 weight, rule:  last playe